In [ ]:
%load_ext autoreload


In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import StandardScaler

%matplotlib inline

pd.set_option('display.max_rows', 200)
pd.set_option('display.max_columns', 200)



In [5]:
y_train = pd.read_parquet("../../dataset/A/train_targets.parquet")
X_train_observed = pd.read_parquet("../../dataset/A/X_train_observed.parquet")
X_train_estimated = pd.read_parquet("../../dataset/A/X_train_estimated.parquet")
X_test = pd.read_parquet("../../dataset/A/X_test_estimated.parquet")
print(f"Observed {X_train_observed.shape}")
print(f"Estimated {X_train_estimated.shape}")
print(f"y {y_train.shape}")
print(f"Test {X_train_observed.shape}")

Observed (118669, 46)
Estimated (17576, 47)
y (34085, 2)
Test (118669, 46)


There is an extra column in estimated 

In [ ]:
set(X_train_estimated.columns) ^ set(X_train_observed.columns)

In [ ]:
X_train_observed.head(100)

In [ ]:
X_train_estimated.head(10)

We shouldn't need date calc at all

In [ ]:
X_train_estimated.drop(["date_calc"], axis=1, inplace=True)
X_train_estimated.shape

In [6]:
y_train.head(100)


,time,pv_measurement
0,2019-06-02 22:00:00,0.00
1,2019-06-02 23:00:00,0.00
2,2019-06-03 00:00:00,0.00
3,2019-06-03 01:00:00,0.00
4,2019-06-03 02:00:00,19.36
5,2019-06-03 03:00:00,251.02
6,2019-06-03 04:00:00,263.78
7,2019-06-03 05:00:00,522.72
8,2019-06-03 06:00:00,904.42
9,2019-06-03 07:00:00,1238.82


In [ ]:
print(y_train.shape[0] * 4)
print(X_train_observed.shape[0] + X_train_estimated.shape[0])
print(y_train.iloc[0,0], y_train.iloc[-1,0])
print(X_train_observed.iloc[0,0], X_train_observed.iloc[-1,0])
print(X_train_estimated.iloc[0,0], X_train_estimated.iloc[-1,0])
print(X_test.iloc[0,0], X_test.iloc[-1,0])


The dimensions of X and y does not match.
We have the weather for every 15 minutes, but doing hourly predictions -> we need to deal with that
For now just repeat the lines of y and then we can average the output
Also there are some missing times in X or Y -> lets delete them, but do not forget, that thy are missing if we need to use the time data

In [ ]:
# y_train = y_train.loc[y_train.index.repeat(4)]
# print(y_train.shape)
# for i in range(int(y_train.shape[0] / 4)):
#     y_train.iloc[4*i+1, 0] += pd.Timedelta("15 min")
#     y_train.iloc[4*i+2, 0] += pd.Timedelta("30 min")
#     y_train.iloc[4*i+3, 0] += pd.Timedelta("45 min")

y_train = y_train.loc[y_train.index.repeat(4)].reset_index(drop=True)
y_train['time'] += y_train.groupby(y_train.index // 4).cumcount() * pd.Timedelta("15 min")



In [ ]:
y_train = y_train[y_train['time'].isin(X_train_observed['date_forecast']) | y_train['time'].isin(X_train_estimated['date_forecast'])].reset_index(drop=True)      
X_train_observed = X_train_observed[X_train_observed['date_forecast'].isin(y_train['time'])].reset_index(drop=True)
X_train_estimated = X_train_estimated[X_train_estimated['date_forecast'].isin(y_train['time'])].reset_index(drop=True)



In [ ]:

print(y_train.shape[0])
print(X_train_observed.shape[0] + X_train_estimated.shape[0])
print(y_train.iloc[0,0], y_train.iloc[-1,0])
print(X_train_observed.iloc[0,0], X_train_observed.iloc[-1,0])
print(X_train_estimated.iloc[0,0], X_train_estimated.iloc[-1,0])

In [ ]:
print(X_train_observed[~ X_train_observed['date_forecast'].isin(y_train['time'])])

In [ ]:
set(X_train_observed.columns[X_train_observed.isna().any()]) & set(X_train_estimated.columns[X_train_estimated.isna().any()].tolist())



These tree columns contains Nans, need make them numbers or remove them

We can put snow density to 0, if there is now snow

In [ ]:
X_train_observed['snow_density:kgm3'] = X_train_observed['snow_density:kgm3'].fillna(0)
X_train_estimated['snow_density:kgm3'] = X_train_estimated['snow_density:kgm3'].fillna(0)



In [ ]:
X_train_observed[X_train_observed['ceiling_height_agl:m'].isna()].head(10)


In [ ]:
X_train_observed[X_train_observed['cloud_base_agl:m'].isna()].shape == X_train_observed[X_train_observed['ceiling_height_agl:m'].isna()].shape


In [ ]:
# Plotting a single feature
fig, axs = plt.subplots(2, 1, figsize=(20, 10), sharex=True)
feature_name = 'cloud_base_agl:m'
X_train_observed[['date_forecast', feature_name]].set_index('date_forecast').plot(ax=axs[0], title=feature_name, color='red')
X_train_estimated[['date_forecast', feature_name]].set_index('date_forecast').plot(ax=axs[0], title=feature_name, color='blue')
X_test[['date_forecast', feature_name]].set_index('date_forecast').plot(ax=axs[0], title=feature_name, color='green')

# Plotting a single feature
feature_name = 'ceiling_height_agl:m'
X_train_observed[['date_forecast', feature_name]].set_index('date_forecast').plot(ax=axs[1], title=feature_name, color='red')
X_train_estimated[['date_forecast', feature_name]].set_index('date_forecast').plot(ax=axs[1], title=feature_name, color='blue')
X_test[['date_forecast', feature_name]].set_index('date_forecast').plot(ax=axs[1], title=feature_name, color='green')


Don't know, how to interpret this, we can use interpolate function

In [ ]:
X_train_observed = X_train_observed.interpolate(axis=0)
X_train_estimated = X_train_estimated.interpolate(axis=0)
# Plotting a single feature
fig, axs = plt.subplots(2, 1, figsize=(20, 10), sharex=True)
feature_name = 'cloud_base_agl:m'
X_train_observed[['date_forecast', feature_name]].set_index('date_forecast').plot(ax=axs[0], title=feature_name, color='red')
X_train_estimated[['date_forecast', feature_name]].set_index('date_forecast').plot(ax=axs[0], title=feature_name, color='blue')
X_test[['date_forecast', feature_name]].set_index('date_forecast').plot(ax=axs[0], title=feature_name, color='green')

# Plotting a single feature
feature_name = 'ceiling_height_agl:m'
X_train_observed[['date_forecast', feature_name]].set_index('date_forecast').plot(ax=axs[1], title=feature_name, color='red')
X_train_estimated[['date_forecast', feature_name]].set_index('date_forecast').plot(ax=axs[1], title=feature_name, color='blue')
X_test[['date_forecast', feature_name]].set_index('date_forecast').plot(ax=axs[1], title=feature_name, color='green')

Let's look to some information about the data

In [ ]:
X_train_observed.describe()

We can see, that snow_drift:idx are only 0. elevation:m is constant, we can remove that too, but we should add a way to distinguish the 3 buildings afterwards (brobably is_a, is_b and is_c columns) if we use all 3 to train 1 model 

In [ ]:
X_train_estimated.drop(columns=["snow_drift:idx", "elevation:m"],  inplace=True)
X_train_observed.drop(columns=["snow_drift:idx", "elevation:m"], inplace=True)


Some of the features like (for example wind) should not be necessary and some could be duplicated - lets look at it
Let's print its correlation with the solution

In [ ]:
display(pd.merge(y_train, X_train_observed.rename(columns={'date_forecast': 'time'}), how="inner", on='time').corr())
with open('correlation.html', 'w') as f: # make it nicer
    print(pd.merge(y_train, X_train_observed.rename(columns={'date_forecast': 'time'}), how="inner", on='time').corr().style.background_gradient()
.to_html(), file=f)


Also lets see, which features would linear regression model use, ass its easy to interpret

In [ ]:
def compute_mne(pred, truth):
    pred[pred < 0] = 0
    mae_nn = np.mean(np.abs(np.array(truth) - pred))
    return mae_nn

In [ ]:
X_train_observed
reg = LinearRegression().fit(X_train_observed.drop("date_forecast", axis=1), y_train[y_train['time'].isin(X_train_observed['date_forecast'])]['pv_measurement'].reset_index(drop=True))
reg.score(X_train_observed.drop("date_forecast", axis=1), y_train[y_train['time'].isin(X_train_observed['date_forecast'])]['pv_measurement'].reset_index(drop=True))

res = reg.predict(X_train_observed.drop("date_forecast", axis=1))
compute_mne(res, y_train[y_train['time'].isin(X_train_observed['date_forecast'])]['pv_measurement'].reset_index(drop=True))

Try this regression use on test data for fun

In [ ]:
X_test.drop(columns=['date_calc', 'snow_drift:idx', 'elevation:m'], inplace=True, errors='ignore')
X_test['snow_density:kgm3'] = X_test['snow_density:kgm3'].fillna(0)
X_test = X_test.interpolate(axis=0)
pred4 = reg.predict(X_test.drop("date_forecast", axis=1))

prediction = np.zeros(pred4.shape[0] // 4)

for i in range(prediction.shape[0]):
    prediction[i] = np.mean(pred4[4*i: 4*(i+1)])

prediction[prediction < 0.] = 0.
prediction = np.hstack([prediction, prediction, prediction])
df = pd.DataFrame({'prediction': prediction})
df.to_csv('easiest_regression.csv', index_label='id')
prediction.shape


In [ ]:
coefs = reg.coef_
#coefs *= np.mean(X_train_observed.drop("date_forecast", axis=1), axis=0) 
for feature, coef in sorted(zip(reg.feature_names_in_, reg.coef_), key=lambda x:abs(x[1]), reverse=True):
    print(feature, coef)

With normalisation


In [ ]:
X_train_observed
regNorm = LinearRegression().fit(StandardScaler().fit_transform(X_train_observed.drop("date_forecast", axis=1)), y_train[y_train['time'].isin(X_train_observed['date_forecast'])]['pv_measurement'].reset_index(drop=True))
regNorm.score(StandardScaler().fit_transform(X_train_observed.drop("date_forecast", axis=1)), y_train[y_train['time'].isin(X_train_observed['date_forecast'])]['pv_measurement'].reset_index(drop=True))

resNorm = regNorm.predict(StandardScaler().fit_transform(X_train_observed.drop("date_forecast", axis=1)))
compute_mne(resNorm, y_train[y_train['time'].isin(X_train_observed['date_forecast'])]['pv_measurement'].reset_index(drop=True))

In [ ]:
#X_test.drop(columns=['date_calc', 'snow_drift:idx', 'elevation:m'], inplace=True)
X_test['snow_density:kgm3'] = X_test['snow_density:kgm3'].fillna(0)
X_test = X_test.interpolate(axis=0)
pred4Norm = regNorm.predict(StandardScaler().fit_transform(X_test.drop("date_forecast", axis=1)))

predictionNorm = np.zeros(pred4.shape[0] // 4)

for i in range(predictionNorm.shape[0]):
    predictionNorm[i] = np.mean(pred4[4*i: 4*(i+1)])

predictionNorm[predictionNorm < 0.] = 0.
predictionNormN = np.hstack([predictionNorm, predictionNorm, predictionNorm])
df = pd.DataFrame({'prediction': predictionNorm})
df.to_csv('easiest_regression.csv', index_label='id')
prediction.shape

In [ ]:

for feature, coef in sorted(zip(reg.feature_names_in_, regNorm.coef_), key=lambda x:abs(x[1]), reverse=True):
    print(feature, coef)


According to the correlation and regression parameters, we are not using wind and snow parameters a lot.
But the snow should be a good indicator of low energy production, we should try to connect it's features into one: is_snow 

All of the preassure features are really high correlated, so we can use only one of them. I choosed the surface preasure

All clear_sky_energy_1h:J and clear_sky_rad:W are also really high correlated, lest drop the 1h value, the same for direct_rad and diffuse_rad

Humidity and dew_point are also highly correlated and I don't think we need them both, let's drop the dew_point_2m:K 

And I am not sure if super_cooled_liquid_water:kgm2 ceiling_height_agl:m cloud_base_agl:m prob_rime:p can have some impact to energy production, but let's assume they do not 


dew_or_rime:idx is column with 1 for dew and -1 for rime, mby we should split this to two indexes

we should split datetime to year, day and hour

and we can add vector of ones, that can help for some regression techniques





Let's make a function, that make's it all also with previous changes

In [2]:
def preprocess(X):
    
    X['snow_density:kgm3'] = X['snow_density:kgm3'].fillna(0)
    X = X.interpolate(axis=0)

    def is_snow(row):
        if row['fresh_snow_24h:cm'] > 0 or row['snow_depth:cm'] > 0:
            return 1
        else:
            return 0
    X['is_snow'] = X.apply(is_snow, axis=1)
    
    def is_dew(row):
        return 1 if row['dew_or_rime:idx'] > 0.1 else 0
    
    def is_rime(row):
        return 1 if row['dew_or_rime:idx'] < -0.1 else 0
    X['is_dew'] = X.apply(is_dew, axis=1)
    X['is_rime'] = X.apply(is_rime, axis=1)
    
    X['year'] = X['date_forecast'].apply(lambda x: x.year)
    X['day_of_year'] = X['date_forecast'].apply(lambda x: x.dayofyear)
    X['hour_of_day'] = X['date_forecast'].apply(lambda x: x.hour)
    
    columns_to_drop = ["snow_drift:idx", "elevation:m", 'snow_melt_10min:mm', 'fresh_snow_12h:cm', 'fresh_snow_3h:cm',  'fresh_snow_6h:cm', 'wind_speed_w_1000hPa:ms', 'snow_water:kgm2', 'snow_density:kgm3', 'fresh_snow_1h:cm', 'fresh_snow_24h:cm',  'wind_speed_v_10m:ms', 'wind_speed_u_10m:ms', 'wind_speed_10m:ms', 'msl_pressure:hPa', 'pressure_100m:hPa', 'pressure_50m:hPa', 'dew_or_rime:idx', 'date_forecast', 'clear_sky_energy_1h:J', 'diffuse_rad_1h:J', 'direct_rad_1h:J', 'dew_point_2m:K']
    columns_to_drop += ['super_cooled_liquid_water:kgm2', 'ceiling_height_agl:m', 'cloud_base_agl:m', 'prob_rime:p']
    
    X.drop(columns=columns_to_drop, inplace=True)
    X.drop(["date_calc"], axis=1, inplace=True, errors='ignore')
    
    X['ones'] = 1
        
    return X

def normalise(X):
    return StandardScaler().fit_transform(X)

def make_y_15mins(y):
    y = y.loc[y.index.repeat(4)].reset_index(drop=True)
    y['time'] += y.groupby(y.index // 4).cumcount() * pd.Timedelta("15 min")
    
    y = y.dropna()
    return y


def match_x_and_y(X, y):
    X = X[X['date_forecast'].isin(y['time'])].reset_index(drop=True)
    y = y[y['time'].isin(X['date_forecast'])].reset_index(drop=True)
    return X, y

def drop_time(y):
    return y.drop('time', axis=1)

def compute_mne(pred, truth):
    pred[pred < 0] = 0
    mae_nn = np.mean(np.abs(np.array(truth) - pred))
    return mae_nn

def mean_15min_output(y):
    y[y < 0.] = 0
    meany = np.zeros(y.shape[0] // 4)
    
    for i in range(meany.shape[0]):
        meany[i] = np.mean(y[4*i: 4*(i+1)])
    
    return meany
    
def create_csv(y, name='model.csv'):
    y[y < 0.] = 0.
    output = pd.DataFrame({'prediction': y})
    output.to_csv(name, index_label='id')
    


And let's make everything again and see, the coeficients now

In [ ]:

y_train = pd.read_parquet("../../dataset/A/train_targets.parquet")
X_train_observed = pd.read_parquet("../../dataset/A/X_train_observed.parquet")

y_train = make_y_15mins(y_train)
X_train_observed, y_train = match_x_and_y(X_train_observed, y_train)
X_train_observed = preprocess(X_train_observed)
y_train = drop_time(y_train)
cols = X_train_observed.columns
#X_train_observed = normalise(X_train_observed)
reg = LinearRegression()
reg.fit(X_train_observed, y_train)
print(reg.score(X_train_observed, y_train))

res = reg.predict(X_train_observed)
print(f'MNE: {compute_mne(resNorm, y_train)} \n')

np.set_printoptions(suppress=True)
print(np.hstack([X_train_observed.iloc[25:50, 1:2], resNorm[25:50,0:1], y_train.iloc[25:50,0:1]]))

coefs = reg.coef_[0]

#coefs*= np.mean(X_train_observed, axis=0) # lets multiply the coefs by values on the coll, so we can see how much impact it has

for feature, coef in sorted(zip(cols, coefs), key=lambda x:abs(x[1]), reverse=True):
    print(feature, coef)
    
    
#print(X_train_observed)

Ok cannot interpret this, but it works somehow, let's try it 

In [3]:
prediction = []

for letter in ['A', 'B', 'C']:
    # read the data
    print(f"dataset {letter}")
    X_train = pd.concat([
        pd.read_parquet(f"../../dataset/{letter}/X_train_observed.parquet"),
        pd.read_parquet(f"../../dataset/{letter}/X_train_estimated.parquet")
    ], ignore_index=True)
    y_train = pd.read_parquet(f"../../dataset/{letter}/train_targets.parquet")
    X_test = pd.read_parquet(f"../../dataset/{letter}/X_test_estimated.parquet")
    print(X_test.shape)
    
    y_train = make_y_15mins(y_train)
    X_train, y_train = match_x_and_y(X_train, y_train)
    y_train = drop_time(y_train)
    X_train = preprocess(X_train)
    X_test = preprocess(X_test)
    
    ML = LinearRegression()
    model = ML.fit(X_train, y_train)   
    if letter == 'A':
        prediction = model.predict(X_test)
    else:
        prediction = np.vstack((prediction, model.predict(X_test)))

prediction = mean_15min_output(prediction)
print(prediction.shape)
prediction[prediction < 0.] = 0. # energy production can't be negative
df = pd.DataFrame({'prediction': prediction.flatten()})
df.to_csv('dropped_columns_regression.csv', index_label='id')
print("done")
    

dataset A
(2880, 47)
dataset B
(2880, 47)
dataset C
(2880, 47)
(2160,)
done


Try the same on different ML than regression, and try to find the best parameters

from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV


y_train = pd.read_parquet("../../dataset/A/train_targets.parquet")
X_train_observed = pd.read_parquet("../../dataset/A/X_train_observed.parquet")
y_test = pd.read_parquet("../../dataset/A/train_targets.parquet")
x_test = pd.read_parquet("../../dataset/A/X_train_estimated.parquet")


y_train = make_y_15mins(y_train)
X_train_observed, y_train = match_x_and_y(X_train_observed, y_train)
X_train_observed = preprocess(X_train_observed)
cols = X_train_observed.columns

y_test = make_y_15mins(y_test)
x_test, y_test = match_x_and_y(x_test, y_test)
x_test = preprocess(x_test)



# Create the parameter grid based on the results of random search 
param_grid = {
    'bootstrap': [True],
    'max_depth': [80, 90, 100, 110],
    'max_features': [2, 3],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [8, 10, 12],
    'n_estimators': [100, 200, 300, 1000]
}

# Create a base model
rf = RandomForestRegressor(random_state = 42)

# Instantiate the grid search model
#grid_search = GridSearchCV(estimator = rf, param_grid = param_grid, 
#                          cv = 3, n_jobs = -1, verbose = 2, return_train_score=True)
#grid_search.fit(X_train_observed, y_train['pv_measurement']);




In [11]:
prediction = []
from sklearn.ensemble import RandomForestRegressor

for letter in ['A', 'B', 'C']:
    # read the data
    print(f"dataset {letter}")
    X_train = pd.concat([
        pd.read_parquet(f"../../dataset/{letter}/X_train_observed.parquet"),
        pd.read_parquet(f"../../dataset/{letter}/X_train_estimated.parquet")
    ], ignore_index=True)
    y_train = pd.read_parquet(f"../../dataset/{letter}/train_targets.parquet")
    X_test = pd.read_parquet(f"../../dataset/{letter}/X_test_estimated.parquet")
    print(X_test.shape)
    
    y_train = make_y_15mins(y_train)
    X_train, y_train = match_x_and_y(X_train, y_train)
    X_train = preprocess(X_train)
    X_test = preprocess(X_test)
    
    ML = RandomForestRegressor(random_state = 42)
    model = ML.fit(X_train, y_train['pv_measurement'])   
    if letter == 'A':
        prediction = model.predict(X_test)
    else:
        prediction = np.concatenate((prediction, model.predict(X_test)))
    

prediction = mean_15min_output(prediction)
print(prediction.shape)
prediction[prediction < 0.] = 0. # energy production can't be negative
df = pd.DataFrame({'prediction': prediction.flatten()})
df.to_csv('dropped_columns_forest.csv', index_label='id')
print("done")

dataset A
(2880, 47)
dataset B
(2880, 47)
dataset C
(2880, 47)
(2160,)
done


We could also try to do one big hour features row instead of meaning four smaller

In [ ]:
y_train = pd.read_parquet("../../dataset/A/train_targets.parquet")
X_train_observed = pd.read_parquet("../../dataset/A/X_train_observed.parquet")

def join_X_hour(X):
    extra_columns = ['hour_of_day', 'day_of_year', 'year']

    X1 = X_train_observed[::4].reset_index()
    X2 = X_train_observed[1::4].reset_index().add_suffix("_+15").drop(columns=extra_columns)
    X3 = X_train_observed[2::4].reset_index().add_suffix("_+30").drop(columns=extra_columns)
    X4 = X_train_observed[3::4].reset_index().add_suffix("_+45").drop(columns=extra_columns)

    return  pd.concat([X1,X2,X3,X4], axis=1)

